# Illustrate Pulling Run Data From ADAM

In [ ]:
from adam import ConfigManager
from adam import ProjectsClient, JobsClient, ResultsClient
from adam import MonteCarloResults
from adam import Comparison
import datetime

This sets up authenticated access to the server. It needs to be done before pretty much everything you want to do with ADAM.

In [ ]:
# ConfigManager loads the config set up via adamctl.
# See the README at https://github.com/B612-Asteroid-Institute/adam_home/blob/master/README.md
config_manager = ConfigManager()
config_manager.set_default_env('dev')
config = config_manager.get_config()

## Getting Projects List

It is also possible to filter the results based on whether the name, description, or project ID contain a desired text value.

In [ ]:
projects = ProjectsClient().get_projects()
print(len(projects))

...or can get the project from your workspace configuration...

In [ ]:
project = ProjectsClient().get_project_from_config(config)
project

## Getting Jobs for project

It is possible to query the service for all the jobs for a project. One can query on fields like: name, earliest/latest submission date, object ID, and most text fields support wildcard searches.

In [ ]:
jobs = JobsClient().get_jobs(project, status="COMPLETED")
for j in jobs:
    print(f"Object: {j.get_object_id()}, Status: {j.get_status()}, Description: {j.get_description()}")

...or get all jobs for the project submitted today...

In [ ]:
today_jobs = JobsClient().get_jobs(project, earliest_submission_datetime=datetime.datetime.today())
for j in today_jobs:
    print(f"Object: {j.get_object_id()}, Time: {j.get_submission_time()}, Description: {j.get_description()}, Status: {j.get_status()}")

...or all jobs submitted last week for a Object ID's contain the word "Asteroid"

In [ ]:
earliest = datetime.datetime.today() - datetime.timedelta(days=30)
latest = datetime.datetime.today() - datetime.timedelta(days=7)
JobsClient().get_jobs(
    project, 
    object_id="*Asteroid*", 
    earliest_submission_datetime=earliest, 
    latest_submission_datetime=latest)

## Post-Filtering Job list

Once you have a job list you can perform additional filtering based on the input JSON data such as:

In [ ]:
JobsClient().filter_by_inputs(jobs, ['monteCarloDraws'], Comparison.GreaterThanOrEquals, 10)

In [ ]:
JobsClient().filter_by_inputs(jobs, ['opm','keplerian','inclination'], Comparison.LessThan, 22.5)

## Get Job Results

Get last job (assuming at least one job) and its inputs JSON

In [ ]:
job = jobs[len(jobs)-1]
print(jobs[0].get_object_id())
print(jobs[0].get_input_json())

### Get Job Result Data

In [ ]:
results = ResultsClient().get_monte_carlo_results(jobs[0])

### Confirm Job Completed

In [ ]:
results.check_status()

### Get Number of Runs

In [ ]:
runs_count = results.get_result_ephemeris_count()
print(f'# Runs: {runs_count}')

### Get Summary Statistics

In [ ]:
stats = results.get_summary()
print(stats)
print(stats.get_misses())

### Get Ephemeris and Graph
It is possible to pull the ephemeris in a way that can be directly used for analysis

In [ ]:
eph = results.get_result_ephemeris(2)
print (eph.values[0])
eph.plot(x='Epoch', y=['X','Y','Z'], ylabel='Position(km)')
eph.plot(x='Epoch', y=['Vx','Vy','Vz'], ylabel="Velocity(km/s)")

### Get Raw Ephemeris Data
It is also possible to pull the raw ephemeris file text as well.

In [ ]:
ephem_raw_data = results.get_result_raw_ephemeris(2)
print(ephem_raw_data)

### Get ending state vector

In [ ]:
end_state_vectors = results.get_final_positions(MonteCarloResults.PositionOrbitType.CLOSE_APPROACH)
print("Close State Vectors")
if len(end_state_vectors) > 0:
    print(f'First close end state: {end_state_vectors[0]}')
else:
    print("None")


end_state_vectors = results.get_final_positions(MonteCarloResults.PositionOrbitType.IMPACT)
print("Impact State Vectors")
if len(end_state_vectors) > 0:
    print(f'First impact end state: {end_state_vectors[0]}')
else:
    print("None")


end_state_vectors = results.get_final_positions(MonteCarloResults.PositionOrbitType.MISS)
print("Miss State Vectors")
if len(end_state_vectors) > 0:
    print(f'First miss end state: {end_state_vectors[0]}')
else:
    print("None")

